In [1]:
import requests
from bs4 import BeautifulSoup as bs
import yaml
from IPython.display import display
from time import sleep
import re
from tqdm import tqdm
import pandas as pd

from typing import List, Dict, Any, Set, Tuple, Union, Optional


In [1]:
import os
os.chdir("..")
print(os.getcwd())

import classes
len(classes.speakings)


c:\Users\이찬행\OneDrive - 충남대학교\문서\GitHub\snu-factcheck-analytics


3292